In [1]:
import pandas as pd
from selenium import webdriver
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from time import sleep
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
from selenium.webdriver.chrome.options import Options

In [2]:
pincodes = pd.read_excel("fsn-pin.xlsx", sheet_name="pincode")
fsn = pd.read_excel("fsn-pin.xlsx",sheet_name="fsn")

In [3]:
fsn = fsn[:10]

In [4]:
a = pincodes.copy()
for i in range(len(fsn)-1):
     a = pd.concat([a,pincodes])

b = fsn.copy()
for i in range(len(pincodes)-1):
    b = pd.concat([b,fsn])

In [5]:
b.sort_index(inplace=True)
a["FSN"] = list(b["FSN"])

In [6]:
a

,Pincode,City,FSN
0,302001,JAIPUR,CAFFCUSYR8ANZKZW
1,390001,VADODARA,CAFFCUSYR8ANZKZW
2,394101,SURAT,CAFFCUSYR8ANZKZW
3,400001,MUMBAI,CAFFCUSYR8ANZKZW
4,410301,PUNE,CAFFCUSYR8ANZKZW
...,...,...,...
29,380005,AHMEDABAD,CAFFCHVGMFWZRRRY
30,382002,Gandhinagar,CAFFCHVGMFWZRRRY
31,521101,KRISHNA,CAFFCHVGMFWZRRRY
32,781001,Guwahati,CAFFCHVGMFWZRRRY


In [7]:
pincode = list(a["Pincode"])
fsn = list(a["FSN"])
city = list(a["City"])

In [8]:
path = r"C:\Users\Sanchit\Downloads\chromedriver.exe"
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')  
options.add_argument('--no-sandbox')
options.add_argument('window-size=1920x1080')

In [9]:
count = 0
df = {"Date":[],"Pincode":[],"City":[],"FSN":[],"Selling Price":[],"Availability":[],"Merchant":[],"Delivery":[],"Stars":[],
     "No. of Ratings":[],"No. of Reviews":[]}

In [12]:

for i in range(42,len(fsn)):
    browser = webdriver.Chrome(executable_path = path, chrome_options = options)
    browser.get("https://www.flipkart.com/yogabar/p/itm2ecc7496c0ad5?pid="+fsn[i])
    sleep(2)
    
    try:
        x = browser.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[3]/div/div[1]/h1/span').text
    except NoSuchElementException:
        x = browser.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[2]/div/div[1]/h1/span').text
    
    if x == "Currently Unavailable":
        df["Selling Price"].append("NA")
        df["Merchant"].append("NA")
        df["Delivery"].append("NA")
        df["Availability"].append("Out of Stock.")
        df["Stars"].append("NA")
        df["No. of Ratings"].append("NA")
        df["No. of Reviews"].append("NA")
    
    else:
        pincode_search_bar = browser.find_element_by_xpath('//*[@id="pincodeInputId"]')  
        pincode_search_bar.send_keys(pincode[i])
        sleep(2)

        pincode_button = browser.find_element_by_class_name("_2P_LDn").click()
        sleep(2)

        price = browser.find_element_by_xpath("//div[@class='_30jeq3 _16Jk6d']").text
        price = price.replace("₹",'')
        price = float(price)
        df["Selling Price"].append(price)

        seller = browser.find_elements_by_xpath("//*[@id='sellerName']/span/span")
        for s in seller:
            x = s.text
        df["Merchant"].append(x)
        
        delivery_by = browser.find_elements_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[4]/div/div/div[2]/div[1]/ul/div/div/span[1]')
#         delivery_by = browser.find_elements_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[5]/div/div/div[2]/div[1]/ul/div/div/span[1]')
        for d in delivery_by:
            x = d.text
        df["Delivery"].append(x)

        availability = browser.find_elements_by_xpath('//*[@id="container"]/div/div[3]/div[4]/div[2]/div[6]/div/div[1]/div[2]/div/ul/div/div[1]/span[1]')
        for i in availability:
            x = i.text
        if x == "Currently out of stock in this area.":
            df["Availability"].append("Out of Stock.")
        else:
            df["Availability"].append("In Stock.")

        star = browser.find_element_by_class_name("_3LWZlK").text
        star = float(star)
        df["Stars"].append(star)
        browser.save_screenshot('check.jpg')
        
        try:
            rating = browser.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[3]/div/div[2]/div/div/span[2]/span/span[1]').text
        except NoSuchElementException:
            try:
                rating = browser.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[2]/div/div[2]/div/div/span[2]/span/span[1]').text
            except NoSuchElementException:
                rating = str(0)
        rating = rating.replace(' &nsb;','')
        rating = rating.replace(' Ratings','')
        rating = rating.replace(',','')
        rating = int(rating)
        df["No. of Ratings"].append(rating)
        
        try:
            review = browser.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[3]/div/div[2]/div/div/span[2]/span/span[3]').text
        except NoSuchElementException:
            try:
                review = browser.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[2]/div/div[2]/div/div/span[2]/span/span[3]').text
            except NoSuchElementException:
                review = str(0)
        review = review.replace(' Reviews','')
        review = review.replace(',','')
        review = review.replace('&nsb;','')
        review = int(review)
        df["No. of Reviews"].append(review)
        
    df["FSN"].append(fsn[i])
    df["Pincode"].append(pincode[i])
    df["City"].append(city[i])
    now = datetime.now()
    df["Date"].append(now)
    
    count+=1
    print(count)
    browser.close()

43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="container"]/div/div[3]/div[1]/div[2]/div[2]/div/div[1]/h1/span"}
  (Session info: headless chrome=102.0.5005.115)


In [ ]:
fsn1 = list(set(fsn))

In [ ]:
fsn1

In [ ]:
mobile_emulation = {
   "deviceMetrics": { "width": 360, "height": 640, "pixelRatio": 3.0 },
   "userAgent": "Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexus 5 Build/JOP40D) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/18.0.1025.166 Mobile Safari/535.19" }

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
options.add_argument('--headless')
options.add_argument('--disable-gpu')  
options.add_argument('--no-sandbox')

last15days = []
count = 0
browser = webdriver.Chrome(path,chrome_options = options)
for i in range(len(fsn1)):  
    browser.get("https://www.flipkart.com/yogabar/p/itm2ecc7496c0ad5?pid="+fsn1[i])
    sleep(2)
    try:
        last15days.append(browser.find_element_by_xpath('//*[@id="_parentCtr_"]/div[5]/div/div/div/div/div[2]/div/div/span[1]').text)
    except NoSuchElementException:
        last15days.append(np.nan)
    count+=1
    print(count)
    print(last15days[-1])
browser.close()

In [ ]:
required = pd.DataFrame(df)
requred1 = pd.DataFrame({"FSN":fsn1,"Last 15 Days Performance":last15days})

In [ ]:
required

In [ ]:
requred1

In [ ]:
required = pd.merge(required,requred1,how="left",on="FSN")

In [ ]:
for i in range(len(required["Delivery"])):
    if required.loc[i,"Delivery"] == 'Yogabar' or required.loc[i,"Delivery"] == 'SuperComNet':
        required.loc[i,"Delivery"] = "NA"

In [ ]:
required

In [ ]:
required.to_excel("fk_sheet_fin1.xlsx")